<a href="https://colab.research.google.com/github/rodchimb/rodchimb/blob/main/Network_Monitoring_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# network_monitor.py

"""
A Python script for real-time network monitoring using the NAPALM library.

NAPALM (Network Automation and Programmability Abstraction Layer with Multivendor support)
provides a unified API to interact with network devices from different vendors,
making it ideal for this task.

This script demonstrates how to connect to Cisco IOS XE and Juniper devices
and retrieve key operational data like device facts, CPU, and memory usage.

Prerequisites:
- Python 3.6+
- pip install napalm
- pip install napalm-junos # For Juniper devices
- Make sure API access (NETCONF for Juniper, RESTCONF for Cisco) is enabled
  on your devices and you have the necessary credentials.
"""

from napalm import get_network_driver
import json
import datetime
import sys
import logging

# Set up basic logging for visibility
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Device Configuration ---
# IMPORTANT: Replace these with your actual device details and credentials.
# NOTE: It's highly recommended to use a secrets management system (e.g., environment
# variables, a vault) for production, rather than hardcoding credentials.

DEVICES = [
    {
        'hostname': 'cisco_iosxe_device_ip',
        'vendor': 'ios',  # NAPALM driver for Cisco IOS XE
        'username': 'your_cisco_username',
        'password': 'your_cisco_password',
        'optional_args': {
            'port': 830, # Default for NETCONF over SSH
            # 'optional_args' can be used to specify different connection parameters
            # for RESTCONF, etc.
        }
    },
    {
        'hostname': 'juniper_device_ip',
        'vendor': 'junos',  # NAPALM driver for Juniper Junos
        'username': 'your_juniper_username',
        'password': 'your_juniper_password',
        'optional_args': {
            'port': 830, # Default for NETCONF over SSH
            # If using SSH, ensure SSH is enabled with NETCONF capabilities
        }
    }
]

# --- Functions for Monitoring ---

def get_device_info(device_info):
    """
    Connects to a single device and retrieves operational data.

    Args:
        device_info (dict): A dictionary containing hostname, vendor, username,
                           and password.

    Returns:
        dict: A dictionary containing the device's hostname and collected data,
              or an error message if the connection fails.
    """
    hostname = device_info['hostname']
    vendor = device_info['vendor']
    username = device_info['username']
    password = device_info['password']
    optional_args = device_info.get('optional_args', {})

    logging.info(f"Attempting to connect to {hostname} ({vendor})...")

    try:
        # Get the appropriate NAPALM driver for the vendor
        driver = get_network_driver(vendor)

        # Create the device object
        device = driver(
            hostname=hostname,
            username=username,
            password=password,
            optional_args=optional_args
        )

        # Open the connection to the device
        device.open()

        # Retrieve a variety of operational data using NAPALM's standard getters
        facts = device.get_facts()
        cpu_metrics = device.get_environment()
        memory_metrics = device.get_environment()

        # Close the connection
        device.close()

        # Parse the data to get the specific metrics we need
        cpu_load = cpu_metrics['cpu'][0]['%usage'] if 'cpu' in cpu_metrics and cpu_metrics['cpu'] else 'N/A'

        # Memory metrics often contain a list of dictionaries; let's sum them or find a key
        memory_total = sum(mem.get('total_ram', 0) for mem in memory_metrics['memory'] if 'total_ram' in mem) if 'memory' in memory_metrics else 'N/A'
        memory_used = sum(mem.get('used_ram', 0) for mem in memory_metrics['memory'] if 'used_ram' in mem) if 'memory' in memory_metrics else 'N/A'
        memory_utilization = (memory_used / memory_total) * 100 if memory_total != 0 else 'N/A'

        # Build the result dictionary
        result = {
            'timestamp': datetime.datetime.now().isoformat(),
            'hostname': hostname,
            'vendor': facts.get('vendor', 'N/A'),
            'os_version': facts.get('os_version', 'N/A'),
            'uptime_seconds': facts.get('uptime', 'N/A'),
            'cpu_utilization_percent': cpu_load,
            'memory_utilization_percent': f"{memory_utilization:.2f}" if isinstance(memory_utilization, float) else memory_utilization
        }

        logging.info(f"Successfully collected data from {hostname}.")
        return result

    except Exception as e:
        logging.error(f"Failed to connect to {hostname}: {e}")
        return {
            'timestamp': datetime.datetime.now().isoformat(),
            'hostname': hostname,
            'status': 'error',
            'error_message': str(e)
        }

def main():
    """
    Main function to orchestrate the monitoring process.
    """
    monitoring_results = []

    # Iterate through the list of devices and monitor each one
    for device_config in DEVICES:
        result = get_device_info(device_config)
        monitoring_results.append(result)

    # Output the results in a human-readable and structured format (JSON)
    print("\n--- Monitoring Report ---")
    print(json.dumps(monitoring_results, indent=2))

    # You could add logic here to:
    # 1. Save this data to a file (e.g., 'monitoring_data.json')
    # 2. Send it to a database (e.g., InfluxDB, Prometheus)
    # 3. Trigger an alert if metrics exceed a threshold

if __name__ == '__main__':
    main()